In [1]:
import Pkg
Pkg.activate("../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("VARs/utils.jl")
includet("bootstrap_model.jl")

In [4]:
#df = DataFrame(XLSX.readtable("../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
df = DataFrame(XLSX.readtable("../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

872×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬─────────┬────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E       │ CAPE  ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼─────────┼────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │    2.44 │ 12.52 ⋯
│ 1952-01-01 │  0.02463 │   104.771 │       23.75 │  1.41667 │ 2.42667 │ 12.36 ⋯
│ 1952-02-01 │  0.02617 │   103.585 │       23.81 │     1.42 │ 2.41333 │ 12.36 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │     2.4 │ 12.24 ⋯
│ 1952-04-01 │  0.02422 │   105.806 │       23.73 │     1.44 │    2.38 │ 12.20 ⋯
│ 1952-05-01 │  0.02384 │   106.373 │       24.38 │     1.45 │    2.36 │ 12.44 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │    2.34 │ 12.66 ⋯
│ 1952-07-01 │  0.02466 │   106.034 │       25.18 │     1.45 │ 2.34667 │ 12.67 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [5]:
#= freq = 12
returns = percentchange(data_source[:ACWI_Net, :USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
#= real_10Y = log1p.(data_source[:Yeld_10Y])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend_Yeld])
term = log1p.(data_source[:Yeld_10Y] ./ freq)  .- log1p.(data_source[:T_Bill] ./ freq)
term = data_source[:Yeld_10Y]   .- log1p.(data_source[:T_Bill]  =#


start = Date(1988, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
#= display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq)) =#

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d))))  =#

In [6]:
freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
yeld_diff = diff(data_source[:Yeld_10Y])
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq)) 

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


...

"Excess_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1234,1.3882,0.0587,0.328,-2.7107,-1.1661,-0.146,0.2279,4.6871,0.4761,0.1029
Price_10Y_T_Bill,0.0753,1.2656,0.0108,0.1488,-1.1145,0.182,-0.1327,0.0984,1.8747,0.1432,-0.0032


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [7]:
display(returns_summarystats(to(from(yeld_diff, start),end_d),1))

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0031,0.0159,0.0,0.0016,-0.0188,-0.3117,-0.0015,0.103,4.0937,0.0061,0.0001


nothing

In [8]:
data = to(from(real_returns, start),end_d)
returns1y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI,],12) ./1
returns5y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],60) ./5
returns10y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],120) ./10
returns30y = moving(sum,data[:Total_Stock_CPI, :Price_10Y_CPI, ],360) ./30
cum_returns = merge(TimeSeries.rename!(returns1y, [:RealStock1y, :RealBonds1y, ]), 
                    TimeSeries.rename!(returns5y, [:RealStock5y, :RealBonds5y, ]),
                    TimeSeries.rename!(returns10y, [:RealStock10y, :RealBonds10y, ]),
                    TimeSeries.rename!(returns30y, [:RealStock30y, :RealBonds30y, ]))
returns_summarystats(expm1.(cum_returns),1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
RealStock1y,0.1568,0.5215,0.0966,0.1997,-0.406,-0.6427,0.0223,0.9394,0.5773,0.6159,0.1163
RealBonds1y,0.0971,0.419,0.0392,0.1007,-0.2337,0.3471,-0.0245,0.938,1.0365,0.4035,0.0378
RealStock5y,0.0731,0.2559,0.0872,0.1342,-0.081,-0.217,0.0452,0.9875,-0.4863,1.1943,0.0977
RealBonds5y,0.0417,0.1962,0.0442,0.0617,-0.0566,0.5776,0.0198,0.9809,1.5522,1.0597,0.0439
RealStock10y,0.0454,0.161,0.0814,0.1125,-0.0591,-0.8527,0.0565,0.9923,0.545,1.7952,0.0909
RealBonds10y,0.0271,0.1129,0.0422,0.0571,-0.0259,-0.0495,0.0278,0.9849,0.3753,1.5574,0.0403
RealStock30y,0.0132,0.0885,0.0664,0.0772,0.04,-0.422,0.0527,0.988,-1.2466,5.0111,0.0707
RealBonds30y,0.0153,0.069,0.0334,0.0428,-0.0015,0.0323,0.0199,0.991,-0.7212,2.189,0.0354


### Stationary bootstrap
- get singe block
- get multople blocs with static length
- get single blocs with random lengt
- get random block length
- gent random block lengt when, lengt is greater when left length

In [9]:
#assets_names = [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
#assets_names = [:Total_Stock_T_Bill, :Price_10Y_T_Bill]
data = to(from(real_returns[assets_names], start),end_d) 
#data = to(from(excess_returns[assets_names], start),end_d) 


835×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1955-01-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 1955-01-01 │       0.0362793 │    -0.0101211 │
│ 1955-02-01 │      -0.0043303 │   -0.00343316 │
│ 1955-03-01 │       0.0372676 │    0.00336866 │
│ 1955-04-01 │    -0.000765351 │   -0.00320268 │
│ 1955-05-01 │       0.0594641 │    0.00504353 │
│ 1955-06-01 │       0.0697441 │    -0.0123327 │
│ 1955-07-01 │     -0.00294578 │    -0.0086779 │
│ 1955-08-01 │       0.0432164 │   0.000622423 │
│     ⋮      │        ⋮        │       ⋮       │
│ 2024-01-01 │       0.0349213 │      -0.01189 │
│ 2024-02-01 │       0.0258185 │    -0.0241356 │
│ 2024-03-01 │      -0.0140089 │     0.0036489 │
│ 2024-04-01 │       0.0231748 │    -0.0372919 │
│ 2024-05-01 │       0.0345217 │     0.0176731 │
│ 2024-06-01 │       0.0233252 │     0.0154399 │
│ 2024-07-01 │      -0.0155933 │     0.025

In [16]:
scenarios = block_bootstrap(values(data), 10_000; block_size=120, random_block=true)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,Price_10Y_CPI
1,0.0602,0.0263
5,0.0604,0.0258
15,0.0596,0.0242
25,0.0601,0.0242


,Total_Stock_CPI,Price_10Y_CPI
1,0.1588,0.0879
5,0.0688,0.0418
15,0.039,0.0266
25,0.0289,0.0209


,Total_Stock_CPI,Price_10Y_CPI
1,-0.8119,0.4679
5,-0.1456,0.2902
15,-0.1587,0.0596
25,-0.1015,0.049


,Total_Stock_CPI,Price_10Y_CPI
1,1.0749,1.0723
5,-0.4466,0.988
15,-0.4897,-0.4494
25,-0.1689,-0.3985


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4429,-0.3215,-0.2272,-0.0285,0.0827,0.1696,0.2775,0.3113,0.3586
5,-0.0906,-0.0697,-0.0505,0.0066,0.0713,0.1096,0.1713,0.1921,0.2115
15,-0.027,-0.0156,-0.0048,0.0303,0.0632,0.0875,0.1216,0.1304,0.1386
25,-0.0092,0.0022,0.0124,0.0403,0.0607,0.0802,0.1072,0.1155,0.1221


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.174,-0.1288,-0.1046,-0.0301,0.0206,0.0792,0.1784,0.2441,0.2918
5,-0.0751,-0.0625,-0.0382,-0.0016,0.024,0.0521,0.0938,0.1174,0.1502
15,-0.0325,-0.0257,-0.0175,0.0034,0.0249,0.0427,0.0689,0.0773,0.0834
25,-0.0204,-0.0154,-0.0096,0.0091,0.0241,0.0392,0.0588,0.0646,0.0712


In [28]:
#assets_names = [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
#assets_names = [:Total_Stock_T_Bill, :Price_10Y_T_Bill]
data = to(from(real_returns[assets_names], start),end_d) 
#data = to(from(excess_returns[assets_names], start),end_d) 

data_mean = mean(data) 
data = data .- values(data_mean)

data = data .+ [.045 .02]


835×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1955-01-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 1955-01-01 │       0.0758095 │    0.00840486 │
│ 1955-02-01 │       0.0351999 │     0.0150928 │
│ 1955-03-01 │       0.0767977 │     0.0218946 │
│ 1955-04-01 │       0.0387648 │     0.0153232 │
│ 1955-05-01 │       0.0989943 │     0.0235695 │
│ 1955-06-01 │        0.109274 │    0.00619325 │
│ 1955-07-01 │       0.0365844 │    0.00984803 │
│ 1955-08-01 │       0.0827466 │     0.0191484 │
│     ⋮      │        ⋮        │       ⋮       │
│ 2024-01-01 │       0.0744515 │    0.00663596 │
│ 2024-02-01 │       0.0653487 │   -0.00560964 │
│ 2024-03-01 │       0.0255213 │     0.0221748 │
│ 2024-04-01 │        0.062705 │    -0.0187659 │
│ 2024-05-01 │       0.0740519 │      0.036199 │
│ 2024-06-01 │       0.0628554 │     0.0339658 │
│ 2024-07-01 │       0.0239369 │     0.043

In [29]:
scenarios = block_bootstrap(values(data), 10_000; block_size=120, random_block=true)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,Price_10Y_CPI
1,0.5352,0.2481
5,0.5345,0.2481
15,0.5338,0.2469
25,0.535,0.2468


,Total_Stock_CPI,Price_10Y_CPI
1,0.1615,0.0885
5,0.0695,0.0423
15,0.039,0.0266
25,0.0289,0.0209


,Total_Stock_CPI,Price_10Y_CPI
1,-0.8376,0.4721
5,-0.1452,0.3348
15,-0.1452,0.0754
25,-0.1293,0.0628


,Total_Stock_CPI,Price_10Y_CPI
1,1.0489,0.9755
5,-0.464,0.8591
15,-0.4925,-0.3511
25,-0.21,-0.3477


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.002,0.1489,0.2393,0.4452,0.5598,0.647,0.7518,0.7862,0.8248
5,0.3838,0.4033,0.4233,0.479,0.5445,0.5842,0.6443,0.6665,0.6858
15,0.4458,0.459,0.4691,0.5048,0.5365,0.5618,0.5959,0.6052,0.6132
25,0.4663,0.4776,0.4869,0.515,0.5356,0.5556,0.5817,0.5894,0.5967


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.0589,0.0932,0.1175,0.1904,0.2431,0.3049,0.4015,0.4656,0.5102
5,0.1474,0.1624,0.1852,0.2198,0.2457,0.2742,0.3174,0.3403,0.3725
15,0.1894,0.1966,0.2054,0.2264,0.2475,0.2651,0.2918,0.2999,0.3068
25,0.2017,0.2073,0.2129,0.2318,0.247,0.2617,0.2811,0.2867,0.2945


In [11]:
quantile(rand(Geometric(1/60), 10_000), [0., .25, .5, .75, 1.])

5-element Vector{Float64}:
   0.0
  17.0
  41.0
  82.0
 498.0

In [12]:
#TODO 
#- compare real returns, excess returns, term i real tbill
#- calc drow down and drow down lengt for scenarios
#- est returns 1955-1980
#- calculate rolling windows (one year, 15 years) percentile
# - compere yeld diffs, inflation from bootstrap

gdy data length 120
n_steps = 60
stsrt_idx = 59

max_end_idx = 1
